# Conversione dei dati di IndicePA in LD

In [4]:
import pandas as pd
import rdflib
import json
from IPython.display import Markdown as md
from IPython.display import JSON

In [5]:
from pybars import Compiler
import urllib.parse

# Funzione helper per evitare di ottenere IRI non valide
def encodeURIComponent(this,value):
    return urllib.parse.quote(value)

def apply_handlebars_template(template, data):
    compiler = Compiler()
    context = {
        'encodeURIComponent': encodeURIComponent
    }
    return compiler.compile(template)(data, helpers=context)


In [9]:
with open("srcdata/data/candidature_enti_ipa.json", 'r') as f:
    indicePA_json=json.load(f)

Per i dati di IndicePA le ontologie di riferimento sono:

* L'ontologia delle organizzazioni https://w3id.org/italia/onto/COV/
* L'ontologia dei luoghi https://w3id.org/italia/onto/Project/
    

In [12]:
md("All'interno del file ci sono {} enti".format(len(indicePA_json["enti"])))

All'interno del file ci sono 236 enti

Il formato di partenza dei dati di IndicePA è:

In [13]:
JSON(indicePA_json["enti"][0])

<IPython.core.display.JSON object>

Definiamo quindi un template per IndicePA

Notare che abbiamo usato l'iterazione all'interno del template e il costrutto `#unless` che permette di generare un JSON-LD valido.

In [33]:
indicePA_template="""
{
    "@context": {
    "CLV": "https://w3id.org/italia/onto/CLV/",
    "COV": "https://w3id.org/italia/onto/COV/",
    "PRJ": "https://w3id.org/italia/onto/Project/",
    "owl": "http://www.w3.org/2002/07/owl#"
    },
    "@graph": [
        {{#each .enti}}
        {
            "@type":"COV:PublicOrganization",
            "@id":"https://w3id.org/italia/data/PublicOrganization/{{encodeURIComponent Codice_IPA}}",
            "owl:sameAs": {
                    "@id": "https://w3id.org/italia/data/PublicOrganization/{{encodeURIComponent Codice_fiscale_ente}}",
                    "@type": "COV:PublicOrganization"
                },
            "COV:hasCategory": {
                "@id":"https://w3id.org/italia/data/PublicOrganizationCategory/{{encodeURIComponent Codice_Categoria}}",
                "@type": "COV:PublicOrganizationCategory"
                },
            "COV:hasPrimaryAddress": {
                "@type": "CLV:Address",
                "CLV:hasCity": {
                    "@type": "CLV:City",
                    "CLV:hasIdentifier": "https://w3id.org/italia/data/identifiers/cities-identifiers/numeric-istat-code/{{Codice_ISTAT}}"
                }
            },
            "COV:IPACode": "{{Codice_IPA}}",
            "COV:legalName": "{{Denominazione_ente}}"
        }
        {{#unless @last}},{{/unless}}
        {{/each}}
    ]
}
"""

In [34]:
# Trasformazione di JSON in JSON-LD

indicePA_json_ld = apply_handlebars_template(indicePA_template,indicePA_json)

In [35]:
with open ("LD/json-ld/candidature_enti_ipa-ld.json", "w+") as f:
    f.write(indicePA_json_ld)

In [36]:
print(indicePA_json_ld)


{
    "@context": {
    "CLV": "https://w3id.org/italia/onto/CLV/",
    "COV": "https://w3id.org/italia/onto/COV/",
    "PRJ": "https://w3id.org/italia/onto/Project/",
    "owl": "http://www.w3.org/2002/07/owl#"
    },
    "@graph": [
        {
            "@type":"COV:PublicOrganization",
            "@id":"https://w3id.org/italia/data/PublicOrganization/c_a087",
            "owl:sameAs": {
                    "@id": "https://w3id.org/italia/data/PublicOrganization/02254070150",
                    "@type": "COV:PublicOrganization"
                },
            "COV:hasCategory": {
                "@id":"https://w3id.org/italia/data/PublicOrganizationCategory/L6",
                "@type": "COV:PublicOrganizationCategory"
                },
            "COV:hasPrimaryAddress": {
                "@type": "CLV:Address",
                "CLV:hasCity": {
                    "@type": "CLV:City",
                    "CLV:hasIdentifier": "https://w3id.org/italia/data/identifiers/cities-iden

In [37]:
indicePA_g = rdflib.Graph()
indicePA_g.parse(data=indicePA_json_ld,format="json-ld");

In [38]:
query="""
SELECT (COUNT(DISTINCT ?CUP) as ?num_CUP) 
WHERE {
    ?prj a PRJ:PublicInvestmentProject;
    PRJ:uniqueProjectCodeValue ?CUP
}
"""

res=opencup_g.query(query)

for row in res:
    print(row)

NameError: name 'opencup_g' is not defined

Come si vede, i CUP sono in numero inferiore, in quanto ve ne sono di duplicati all'interno del file.

Esempio di query: qual è il totale di costo dei progetti per ciascun ente?

In [24]:
query="""
SELECT ?funder (SUM(?totalCost) AS ?funderTotalCost)
WHERE {
    ?prj a PRJ:PublicInvestmentProject;
    PRJ:projectFunder ?funder;
    PRJ:projectTotalCost ?totalCost.
}
GROUP BY ?funder
ORDER BY DESC(?funderTotalCost)
"""

res=opencup_g.query(query)

for row in res:
    print(row)

(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/00339370272'), rdflib.term.Literal('7127677', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/80015010723'), rdflib.term.Literal('4858680', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/00297110389'), rdflib.term.Literal('1692369', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/00080270838'), rdflib.term.Literal('1547897', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/00229080338'), rdflib.term.Literal('1479027', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/Publi

## Dati di IndicePA

I dati di IndicePA ci occorrono, ad esempio, per collegare le partite IVA/Codici Fiscali agli enti